In [3]:
import torch
import torch.nn as nn

In [4]:
class Generator(nn.Module):
    def __init__(self, nz=50, ngf=[512, 256, 128, 64], nc=3):
        super().__init__()
        self.net = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf[0], 5, stride=2, padding=2, output_padding=1),
            nn.BatchNorm2d(ngf[0]),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf[0], ngf[1], 5, stride=2, padding=2, output_padding=1),
            nn.BatchNorm2d(ngf[1]),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf[1], ngf[2], 5, stride=2, padding=2, output_padding=1),
            nn.BatchNorm2d(ngf[2]),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf[2], ngf[3], 5, stride=2, padding=2, output_padding=1),
            nn.BatchNorm2d(ngf[3]),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf[3], nc, 5, stride=2, padding=2, output_padding=1),
            nn.Tanh()
        )
        # --- Version généralisée équivalente ---
        layers = []
        in_channels = nz 

        # Main Blocs (ConvT -> BatchNorm -> ReLU)
        for out_channels in ngf:
            layers.extend([
                nn.ConvTranspose2d(in_channels, out_channels, 5, stride=2, padding=2, output_padding=1),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(True)
            ])
            in_channels = out_channels 

        # last bloc (ConvT -> Tanh)
        layers.append(nn.ConvTranspose2d(in_channels, nc, 5, stride=2, padding=2, output_padding=1))
        layers.append(nn.Tanh())

        self.net2 = nn.Sequential(*layers)

    def forward(self, z):
        return self.net(z)

In [5]:
gen = Generator()
print(gen.net)
print(gen.net2)

Sequential(
  (0): ConvTranspose2d(50, 512, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1))
  (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): ConvTranspose2d(512, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1))
  (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU(inplace=True)
  (6): ConvTranspose2d(256, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1))
  (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU(inplace=True)
  (9): ConvTranspose2d(128, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1))
  (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): ReLU(inplace=True)
  (12): ConvTranspose2d(64, 3, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1))


In [12]:
str(gen.net) == str(gen.net2) 

True

In [15]:
class Discriminator(nn.Module):
    def __init__(self, nc=3, ndf=[64, 128, 256, 512]):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(nc, ndf[0], 5, stride=2, padding=2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf[0], ndf[1], 5, stride=2, padding=2),
            nn.BatchNorm2d(ndf[1]),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf[1], ndf[2], 5, stride=2, padding=2),
            nn.BatchNorm2d(ndf[2]),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf[2], ndf[3], 5, stride=2, padding=2),
            nn.BatchNorm2d(ndf[3]),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf[3], 1, 5, stride=2, padding=2),
            nn.Sigmoid()
        )
        # --- Version généralisée équivalente ---
        # Premiers layers (Sans BatchNorm
        layers = [nn.Conv2d(nc, ndf[0], 5, stride=2, padding=2),
                nn.LeakyReLU(0.2, inplace=True),]
        in_channels = ndf[0]

        # Main Blocs (Conv -> BatchNorm -> LeakyReLU)
        for out_channels in ndf[1:]:
            layers.extend([
                nn.Conv2d(in_channels, out_channels, 5, stride=2, padding=2),
                nn.BatchNorm2d(out_channels),
                nn.LeakyReLU(0.2, inplace=True),
            ])
            in_channels = out_channels 

        # last bloc (Conv -> Sigmoid)
        layers.append(nn.Conv2d(in_channels, 1, 5, stride=2, padding=2))
        layers.append(nn.Sigmoid())

        self.net2 = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

In [16]:
dis = Discriminator()
print(dis.net)
print(dis.net2)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (1): LeakyReLU(negative_slope=0.2, inplace=True)
  (2): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): LeakyReLU(negative_slope=0.2, inplace=True)
  (5): Conv2d(128, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): LeakyReLU(negative_slope=0.2, inplace=True)
  (8): Conv2d(256, 512, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): LeakyReLU(negative_slope=0.2, inplace=True)
  (11): Conv2d(512, 1, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (12): Sigmoid()
)
Sequential(
  (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (1): LeakyReLU(negative_slope=0.2, inplace=

In [17]:
str(dis.net) == str(dis.net2) 

True